| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

# Working with Mineral Composition Data


### Mineral Database
pyrolite includes a limited mineral database which is useful for looking up endmember compositions.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pyrolite.mineral.mindb import (
    list_groups,
    list_minerals,
    list_formulae,
    get_mineral,
    get_mineral_group,
)

pd.options.display.precision = 3  # smaller graphical outputs

From the database, you can get the list of its contents using a few utility
functions:



In [ ]:
list_groups()

In [ ]:
", ".join(
    list_minerals()
)  # string the list of minerals together so we can print it succinctly

In [ ]:
", ".join(
    list_formulae()
)  # string the list of minerals formulae so we can print it succinctly

You can also directly get the composition of specific minerals by name. These will be a `pd.Series` of elemental Wt% data (although derived from the formuale directly), and normalised to 100%:

In [ ]:
get_mineral("forsterite")

If you want to get compositions for all minerals within a specific group, you can
use `get_mineral_group`:



In [ ]:
get_mineral_group("olivine")

In some cases, these might give some useful reference points for comparing your mineral or whole-rock data to:

In [ ]:
px_comps = get_mineral_group("pyroxene").pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "CaO", "FeO", "MnO", "Al2O3", "Na2O"]
)
px_comps.iloc[:, 2:] = (
    px_comps.iloc[:, 2:].fillna(0) * 100
)  # pyroxene compositions as Wt% oxides
ax = px_comps[["CaO", "MgO"]].pyroplot.scatter(c="k", s=10)

If you wanted to, you could also label these (noting there's some overlap, we here join the names for each point):

In [ ]:
from collections import defaultdict

ax = px_comps[["CaO", "MgO"]].pyroplot.scatter(c="k", s=10)
# round to 0 to account for *nearly* overlapping points
_points = px_comps.round(0).set_index(["CaO", "MgO"])
for position in _points.index.unique():
    minerals = _points.loc[position]
    ax.annotate(
        "\n".join(minerals["name"]),
        xy=position,
        xytext=(5, 0),
        textcoords="offset points",
        va="bottom",
    )


In [ ]:
get_mineral_group("pyroxene")

In [ ]:
px_comps[["CaO", "MgO", "Al2O3"]].pyroplot.scatter(
    c="r", marker="o", s=150, zorder=10, facecolors="none"
)


----
**Optional:** Check out some of the other minerals and mineral groups.

In [ ]:
from pyrolite.mineral.mindb import get_mineral_group, list_groups, list_minerals

list_groups()

In [ ]:
get_mineral_group('olivine')

----
<div class='alert alert-warning'> <font size="+1" color="black"><b> Checkpoint & Time Check</b><br>How are things going?</font></div>


<div class='alert alert-success'><font size="+1" color="black"><b>We made it to the end of the core <code>pyrolite</code> section - there are a few 'workflow' notebooks to have a look at though! </b></font> </div>

----

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |